In [2]:
#########################################load model###############################
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("************ NOTE: The torch device is:", device)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1,16,3,2,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Conv2d(16,32,3,2,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Conv2d(32,64,3,2,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Conv2d(64,128,3,2,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Conv2d(128,256,3,2,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(12* 21* 256,100),
            #nn.BatchNorm2d(128),
            nn.ReLU(),
            #nn.Dropout(0.2)
        )
        
        self.fc = nn.Sequential( 
            nn.Linear(100,12* 21* 256),
            #nn.BatchNorm2d(6*10*128),
            nn.ReLU(),
            #nn.Dropout(0.2)
        )
        
        self.decoder = nn.Sequential( 
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), 
            nn.Sigmoid()
        )


    def forward(self, anchor, positive, negative):
        embedded_anchor   = self.encoder(anchor)
        embedded_positive = self.encoder(positive)
        embedded_negative = self.encoder(negative)
        
        input_to_decoder_anchor   = self.fc(embedded_anchor).view(-1,256,12,21)
        input_to_decoder_positive = self.fc(embedded_anchor).view(-1,256,12,21)
        input_to_decoder_negative = self.fc(embedded_anchor).view(-1,256,12,21)
        
        decoded_anchor   = self.decoder(input_to_decoder_anchor)
        decoded_positive = self.decoder(input_to_decoder_positive)
        decoded_negative = self.decoder(input_to_decoder_negative)
        
        return decoded_anchor, decoded_positive, decoded_negative, embedded_anchor, embedded_positive, embedded_negative
        
    def get_embeding(self,x):
        return self.encoder(x)
        
model = Net()
model = model.to(device)
epoch=67
model.load_state_dict(torch.load( '../../model/img/alpha_1/emb_100/model/{}.pth'.format(epoch) , map_location=device))
######################################

************ NOTE: The torch device is: cpu


<All keys matched successfully>

In [9]:
print("start the code")
import numpy as np
from bob.bio.vein.configurations.utfvp import database
import bob.io.base

from bob.bio.vein.preprocessor import NoCrop, TomesLeeMask, HuangNormalization, \
    NoFilter, Preprocessor

preprocessor = Preprocessor(
    crop=NoCrop(),
    mask=TomesLeeMask(),
    normalize=HuangNormalization(),
    filter=NoFilter(),
    )


from bob.bio.vein.extractor import WideLineDetector
extractor = WideLineDetector()


all_Files = database.objects(protocol='nom', groups='world')
    
######################################################################    
import time 
all_time=[]

for i,obj in enumerate(all_Files[:100]):
    path=obj.make_path('/idiap/temp/hotroshi/UTFVP/data','.png')
    image = bob.io.base.load(path)
    
    image_and_mask = preprocessor(image)
    
    t1= time.time()
    finger_image = image# image_and_mask[0]   
    finger_mask = image_and_mask[1]
    finger_image_ = np.zeros([1,1,finger_image.shape[0]+4,finger_image.shape[1]])
    finger_image_[0,0,2:-2,:] = finger_image[:,:]/255.0
    finger_image_torch = torch.tensor(finger_image_, requires_grad=False).float().to(device)
    feature = model.get_embeding(finger_image_torch).cpu().detach().numpy()
        
    all_time.append(time.time()-t1)
    
print(np.mean(all_time[-1]))

start the code
preparing data
0.05680227279663086
(1, 100)


/idiap/temp/hotroshi/anaconda3/envs/bob_env_pytorch/lib/python3.7/site-packages/bob/bio/vein/preprocessor/normalize.py:137: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = numpy.linalg.lstsq(A,bl)[0] # obtaining the parameters


0.058064937591552734
(1, 100)
0.05633234977722168
(1, 100)
0.05682253837585449
(1, 100)
0.06197047233581543
(1, 100)
0.05721426010131836
(1, 100)
0.0575101375579834
(1, 100)
0.05710887908935547
(1, 100)
0.057004451751708984
(1, 100)
0.05729341506958008
(1, 100)
0.0570068359375
(1, 100)
0.05746054649353027
(1, 100)
0.0571897029876709
(1, 100)
0.057146310806274414
(1, 100)
0.057524681091308594
(1, 100)
0.05712413787841797
(1, 100)
0.05710029602050781
(1, 100)
0.06477642059326172
(1, 100)
0.06372356414794922
(1, 100)
0.06109976768493652
(1, 100)
0.05744528770446777
(1, 100)
0.05694270133972168
(1, 100)
0.0570986270904541
(1, 100)
0.05686330795288086
(1, 100)
0.05707859992980957
(1, 100)
0.05698800086975098
(1, 100)
0.05704069137573242
(1, 100)
0.05718493461608887
(1, 100)
0.05694913864135742
(1, 100)
0.0569915771484375
(1, 100)
0.05692887306213379
(1, 100)
0.057051658630371094
(1, 100)
0.057439565658569336
(1, 100)
0.05722188949584961
(1, 100)
0.057209014892578125
(1, 100)
0.0574240684509

KeyboardInterrupt: 